In [31]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import Markdown as md
from ipynb.fs.full import functions
import pandas as pd

InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [32]:
import yfinance as yf

In [33]:
ticker_df = pd.read_excel('tickers.xlsx', sheet_name=0, index_col=0)

In [34]:
ticker_list = ticker_df.index.to_list()
ticker_list = [f'{ticker}.JK' for ticker in ticker_list]
start_date = '2019-07-01'
end_date = '2023-08-31'

download_result = yf.download(tickers=ticker_list, start=start_date, end=end_date, interval='1wk', threads=False, repair=True, 
                                  keepna=False, period=None)
price_result = download_result.copy()['Close']

[*********************100%%**********************]  425 of 425 completed


In [61]:
price_result.columns = price_result.copy().columns.str.replace('.JK', '')

ticker_return = price_result.copy().ffill().pct_change()[1:]

sector_dict = ticker_df.copy().groupby('sector').groups
sector_return_dict = {}

for sector in sector_dict.keys():
    sector_list = sector_dict[sector]
    sector_return = ticker_return[sector_list].mean(axis=1).to_list()
    sector_return_dict[sector] = sector_return

sector_return_df = pd.DataFrame(sector_return_dict)
sector_corr_df = sector_return_df.corr()

referred_sector = 'basic-materials'
referred_corr = sector_corr_df[referred_sector].sort_values(ascending=False)

referred_corr


basic-materials             1.000000
infrastructures             0.794953
consumer-non-cyclicals      0.752307
energy                      0.719106
industrials                 0.675934
properties-real-estate      0.671995
consumer-cyclical           0.651580
healthcare                  0.595391
transaportation-logistic    0.442675
technology                  0.396241
financials                  0.258612
Name: basic-materials, dtype: float64

In [62]:
with pd.ExcelWriter('gathered_cor.xlsx') as excel_writer:
    referred_corr.to_excel(excel_writer=excel_writer)